In [9]:
import torch
import torchtext.vocab as vocab

In [10]:
vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [11]:
[key for key in vocab.pretrained_aliases.keys()
        if "glove" in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [12]:
cache_dir = "../data"
# glove = vocab.pretrained_aliases["glove.6B.50d"](cache=cache_dir)
glove = vocab.GloVe(name='6B', dim=50, cache=cache_dir) # 与上面等价

 99%|█████████▉| 395949/400000 [00:08<00:00, 45421.45it/s]


In [15]:
print("一共包含%d个词。" % len(glove.stoi))

一共包含400000个词。


In [16]:
glove.stoi['beautiful'], glove.itos[3366] # (3366, 'beautiful')

(3366, 'beautiful')

In [18]:
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x.view((-1,))) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]


In [19]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,
                    embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))


In [25]:
get_similar_tokens('dog', 3, glove)

cosine sim=0.922: cat
cosine sim=0.851: dogs
cosine sim=0.791: horse


In [26]:
get_similar_tokens('baby', 3, glove)


cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [27]:
get_similar_tokens('beautiful', 3, glove)


cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


In [28]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]] 
                for t in [token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos[topk[0]]

In [32]:
get_analogy('man', 'woman', '', glove) # 'daughter'

'she'

In [35]:
get_analogy('beijing', 'china', 'tokyo', glove) # 'japan'


'japan'

In [36]:
get_analogy('bad', 'worst', 'big', glove) # 'biggest'


'biggest'

In [37]:
get_analogy('do', 'did', 'go', glove) # 'went'


'went'